In [4]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf 

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

train_url = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle/3004/train.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1528608739&Signature=KYTkqJ%2B2XjGrrc7%2FglFccTd6LLAyKniT1UISKjowXHRUWpTThU42i5ZL%2BFfiYcDPrQRSZHS8bB5nz86iOcax%2BkS97E1GGfZkRgQn0nC6EnotbqXEgwHBrMxFR9bQfk1i%2FXghulKH%2BuJdqJMXFhocuP2usO7h%2B5D8gzl14rZvOiBminBYwN9JTRTxmN6ugxfN14jm5M%2B%2FZk%2BL4odOiD%2BZ%2BmmmNBfPw%2F9%2FKa6UJLSRWKZA4SrRw0enNxahmAKbgvRV0Kc7YOMIzp0glckFGi%2Fhe2EkPIOibA0EajIKfrnU%2FQrhS8RJy3tHkjfBogy8mNOPf%2BJkrZCp3O3OK56UnmTFYQ%3D%3D'
test_url = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle/3004/test.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1528608707&Signature=swcilvB6CABwyekMsW0oc84NSIy0TGarWrObvLfmpwOPi%2FkFsbeih9AP%2FNbBzDXw6ltx615WYUt%2FZhq8qdovyIN6amKz%2BLbb3Ky2NrIJ5ys6uf2Hu7Qe4mBGV3Q4iDaZ1zsW9E8QCQDWaaabFSl8erMCdHe56PLXLx8CgRWBNbnIOMtUDk0t6lwiXYKFisLyo8KSIyJAsxNF%2ByZXquI%2FYuBXCxFjIzAH2OHoHHt3S1ZK4qSBmx9PhpKA%2B6HH%2BckShvLbGLp%2FHaJ%2FNPWZNYEr%2FHeAQp%2BbCHC5P9XQSB%2Fs7uI%2BzcbO4BiEdU%2Bn93daVlqq2uE%2BWZaTVVPbbFAF06lSPw%3D%3D'

train_df = pd.read_csv(train_url) 
test_df = pd.read_csv(test_url)

In [ ]:
def print_accuracy(clf):
    X = train_df[train_df.columns[1:]]
    Y = train_df['label']
    
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state=42)
    train_sample_size = len(x_train)
    x_train = x_train[x_train.columns][:train_sample_size]
    y_train = y_train[:train_sample_size]
    clf.fit(x_train, y_train)

    print("train accuracy: {:.2%}, test accuracy: {:.2%}".format(
        accuracy_score(y_train, clf.predict(x_train)),
        accuracy_score(y_test, clf.predict(x_test))
    ))
hidden_layer_size_lst = [
    [10, 10], 
    [10, 20, 10],
    [10, 80, 240, 80, 10],
    [10, 20, 40, 80, 40, 20, 10],
    [10, 20, 40, 80, 160, 80, 40, 20, 10],
    [10, 80, 240, 960, 240, 80, 10],
    [10, 80, 240, 480, 960, 480, 240, 80, 10],
]
for h_l_s in hidden_layer_size_lst:
    print("hidden_layer_size: {}".format(h_l_s))
    print_accuracy( MLPClassifier(hidden_layer_sizes=h_l_s, activation='relu', solver='adam') )
    print("-----------------")


hidden_layer_size: [10, 10]
train accuracy: 90.22%, test accuracy: 87.17%


/Users/frhyme/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


-----------------
hidden_layer_size: [10, 20, 10]
train accuracy: 65.79%, test accuracy: 64.71%
-----------------
hidden_layer_size: [10, 80, 240, 80, 10]
train accuracy: 77.44%, test accuracy: 76.98%
-----------------
hidden_layer_size: [10, 20, 40, 80, 40, 20, 10]
train accuracy: 93.27%, test accuracy: 90.69%
-----------------
hidden_layer_size: [10, 20, 40, 80, 160, 80, 40, 20, 10]
train accuracy: 95.58%, test accuracy: 93.10%
-----------------
hidden_layer_size: [10, 80, 240, 960, 240, 80, 10]


In [5]:
"""
- keras에서는 Y를 one_hot vector로 바꾸어 사용해야 함
- X의 값들이 0과 256 사이에 분포해있으므로, 이를 0과 1.0 사이로 움직임 
- train, test set으로 구분하여 진행
"""
X = train_df[train_df.columns[1:]]
Y = pd.get_dummies(train_df['label'])
x_train, x_test, y_train, y_test = train_test_split(X.values.astype(np.float64)/256.0, 
                                                    Y.values.astype(np.float32), 
                                                    test_size = 0.2, random_state=42)

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam, SGD
from keras import metrics
"""
- 원래 데이터가 784이므로, 비슷한 1024로 두고 이를 감소시키는 뉴럴넷 설계
- 총 10개로 클래스를 구분하므로, 마지막 Dense는 10개의 노드를 가지고 있어야함
- 그리고, 마지막은 softmax
"""
model = Sequential([
    Dense(1024, input_shape=(784,)),
    Activation('relu'),
    Dense(512),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(128),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])
"""
- multi classification이므로 loss는 'categorical_crossentropy'
- metric에는 내가 추적할 지표들이 들어감. 최적화는 loss에 따라 되는데, epoch 마다 평가될 지표들이 metric에 들어감. 
"""
model.compile(loss='categorical_crossentropy', 
              #optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True), 
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
              metrics=[metrics.categorical_accuracy])
"""
- one epoch = one forward pass and one backward pass of all the training examples
- batch size = the number of training examples in one forward/backward pass. 

- 즉, epoch을 증가한다는 것은 전체 데이터 셋을 몇 번 돌릴 것이냐 를 결정하는 것이고, 
- batch_size는 backpropagation을 몇 개의 size로 돌릴 것이냐? 를 결정하는 이야기다.
"""
train_history = model.fit(x_train, y_train, epochs=20, batch_size=500, verbose=2)
train_history = train_history.history # epoch마다 변화한 loss, metric

# 아래 세 줄은 필요없는 코드인데, 그래도 이후에 사용될 수 있어서 일단 넣어둠. 
y_predict = model.predict_classes(x_train, verbose=0)
y_true = [ np.argmax(y) for y in y_train]
accuracy = np.sum([y_comp[0]==y_comp[1] for y_comp in zip(y_predict, y_true)])

loss_and_metric = model.evaluate(x_train, y_train, batch_size=128, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))
loss_and_metric = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
print("test, loss and metric: {}".format(loss_and_metric))

Epoch 1/20
11s - loss: 0.5017 - categorical_accuracy: 0.8496
Epoch 2/20
10s - loss: 0.1454 - categorical_accuracy: 0.9573
Epoch 3/20
9s - loss: 0.0880 - categorical_accuracy: 0.9742
Epoch 4/20
9s - loss: 0.0604 - categorical_accuracy: 0.9822
Epoch 5/20
9s - loss: 0.0405 - categorical_accuracy: 0.9873
Epoch 6/20
9s - loss: 0.0294 - categorical_accuracy: 0.9910
Epoch 7/20
9s - loss: 0.0240 - categorical_accuracy: 0.9920
Epoch 8/20
9s - loss: 0.0163 - categorical_accuracy: 0.9949
Epoch 9/20
9s - loss: 0.0148 - categorical_accuracy: 0.9951
Epoch 10/20
9s - loss: 0.0161 - categorical_accuracy: 0.9949
Epoch 11/20
10s - loss: 0.0122 - categorical_accuracy: 0.9959
Epoch 12/20
11s - loss: 0.0083 - categorical_accuracy: 0.9975
Epoch 13/20
12s - loss: 0.0069 - categorical_accuracy: 0.9978
Epoch 14/20
11s - loss: 0.0027 - categorical_accuracy: 0.9992
Epoch 15/20
12s - loss: 0.0086 - categorical_accuracy: 0.9972
Epoch 16/20
11s - loss: 0.0170 - categorical_accuracy: 0.9949
Epoch 17/20
9s - loss: 0.

In [3]:
"""
- keras에서는 Y를 one_hot vector로 바꾸어 사용해야 함
- X의 값들이 0과 256 사이에 분포해있으므로, 이를 0과 1.0 사이로 움직임 
- train, test set으로 구분하여 진행
"""
X = train_df[train_df.columns[1:]]
Y = pd.get_dummies(train_df['label'])

X_values = (X.values.astype(np.float64)/256.0).reshape(len(X), 28, 28, 1)
Y_values = Y.values.astype(np.float32)

x_train, x_test, y_train, y_test = train_test_split(X_values, Y_values, 
                                                    test_size = 0.2, random_state=42)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import SGD
from keras import metrics
import numpy as np

model = Sequential([
    Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(64, (5, 5), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), 
              metrics=[metrics.categorical_accuracy])

train_history = model.fit(x_train, y_train, epochs=10, batch_size=500, verbose=1)
train_history = train_history.history

loss_and_metric = model.evaluate(x_train, y_train, batch_size=128, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))
loss_and_metric = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
print("test, loss and metric: {}".format(loss_and_metric))

NameError: name 'train_test_split' is not defined

In [328]:
X = train_df[train_df.columns[1:]]
Y = pd.get_dummies(train_df['label'])

X_values = (X.values.astype(np.float64)/256.0).reshape(len(X_values), 28, 28, 1)
Y_values = Y.values.astype(np.float32)

x_train, x_test, y_train, y_test = train_test_split(X_values, Y_values, 
                                                    test_size = 0.2, random_state=42)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import SGD
from keras import metrics
import numpy as np

model = Sequential([
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)),
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2)),
    #Dropout(0.25),
    
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    #Dropout(0.25),
    
    Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    Flatten(),
    Dense(256, activation = "relu"),
    #Dropout(0.5),
    Dense(10, activation = "softmax")
])

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), 
              metrics=[metrics.categorical_accuracy])

train_history = model.fit(x_train, y_train, epochs=10, batch_size=500, verbose=1)
train_history = train_history.history

loss_and_metric = model.evaluate(x_train, y_train, batch_size=128, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))
loss_and_metric = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
print("test, loss and metric: {}".format(loss_and_metric))

Epoch 1/10
33600/33600 [==============================] - 504s - loss: 0.5980 - categorical_accuracy: 0.8100   
Epoch 2/10
33600/33600 [==============================] - 481s - loss: 0.0938 - categorical_accuracy: 0.9699   
Epoch 3/10
33600/33600 [==============================] - 462s - loss: 0.0650 - categorical_accuracy: 0.9795   
Epoch 4/10
33600/33600 [==============================] - 478s - loss: 0.0479 - categorical_accuracy: 0.9854   
Epoch 5/10
33600/33600 [==============================] - 462s - loss: 0.0354 - categorical_accuracy: 0.9884   
Epoch 6/10
33600/33600 [==============================] - 438s - loss: 0.0271 - categorical_accuracy: 0.9913   
Epoch 7/10
33600/33600 [==============================] - 416s - loss: 0.0263 - categorical_accuracy: 0.9912   
Epoch 8/10
33600/33600 [==============================] - 442s - loss: 0.0209 - categorical_accuracy: 0.9929   
Epoch 9/10
33600/33600 [==============================] - 474s - loss: 0.0223 - categorical_accuracy: 0.

In [332]:
X = train_df[train_df.columns[1:]]
Y = pd.get_dummies(train_df['label'])

X_values = (X.values.astype(np.float64)/256.0).reshape(len(X_values), 28, 28, 1)
Y_values = Y.values.astype(np.float32)

x_train, x_test, y_train, y_test = train_test_split(X_values, Y_values, 
                                                    test_size = 0.2, random_state=42)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import SGD
from keras import metrics
import numpy as np

model = Sequential([
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)),
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2)),
    #Dropout(0.25),
    
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    #Dropout(0.25),
    
    Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    Flatten(),
    Dense(256, activation = "relu"),
    #Dropout(0.5),
    Dense(10, activation = "softmax")
])

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), 
              metrics=[metrics.categorical_accuracy])

train_history = model.fit(x_train, y_train, epochs=20, batch_size=500, verbose=1)
train_history = train_history.history

loss_and_metric = model.evaluate(x_train, y_train, batch_size=128, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))
loss_and_metric = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
print("test, loss and metric: {}".format(loss_and_metric))

Epoch 1/20
33600/33600 [==============================] - 526s - loss: 0.5985 - categorical_accuracy: 0.8046   
Epoch 2/20
33600/33600 [==============================] - 482s - loss: 0.0885 - categorical_accuracy: 0.9729   
Epoch 3/20
33600/33600 [==============================] - 10337s - loss: 0.0556 - categorical_accuracy: 0.9826  
Epoch 4/20
33600/33600 [==============================] - 2295s - loss: 0.0458 - categorical_accuracy: 0.9856  
Epoch 5/20
33600/33600 [==============================] - 396s - loss: 0.0366 - categorical_accuracy: 0.9883   
Epoch 6/20
33600/33600 [==============================] - 384s - loss: 0.0277 - categorical_accuracy: 0.9914   
Epoch 7/20
33600/33600 [==============================] - 383s - loss: 0.0215 - categorical_accuracy: 0.9934   
Epoch 8/20
33600/33600 [==============================] - 379s - loss: 0.0216 - categorical_accuracy: 0.9925   
Epoch 9/20
33600/33600 [==============================] - 363s - loss: 0.0192 - categorical_accuracy: 0

In [335]:
X = train_df[train_df.columns[1:]]
Y = pd.get_dummies(train_df['label'])

X_values = (X.values.astype(np.float64)/256.0).reshape(len(X_values), 28, 28, 1)
Y_values = Y.values.astype(np.float32)

x_train, x_test, y_train, y_test = train_test_split(X_values, Y_values, 
                                                    test_size = 0.2, random_state=42)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import SGD
from keras import metrics
import numpy as np

model = Sequential([
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)),
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2)),
    #Dropout(0.25),
    
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    #Dropout(0.25),
    
    Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    Flatten(),
    Dense(256, activation = "relu"),
    #Dropout(0.5),
    Dense(10, activation = "softmax")
])

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), 
              metrics=[metrics.categorical_accuracy])

train_history = model.fit(x_train, y_train, epochs=20, batch_size=500, verbose=1)
train_history = train_history.history

loss_and_metric = model.evaluate(x_train, y_train, batch_size=128, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))
loss_and_metric = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
print("test, loss and metric: {}".format(loss_and_metric))

Epoch 1/20
33600/33600 [==============================] - 470s - loss: 0.9917 - categorical_accuracy: 0.6965   
Epoch 2/20
33600/33600 [==============================] - 433s - loss: 0.1283 - categorical_accuracy: 0.9590   
Epoch 3/20
33600/33600 [==============================] - 423s - loss: 0.0739 - categorical_accuracy: 0.9766   
Epoch 4/20
33600/33600 [==============================] - 424s - loss: 0.0603 - categorical_accuracy: 0.9813   
Epoch 5/20
33600/33600 [==============================] - 417s - loss: 0.0408 - categorical_accuracy: 0.9873   
Epoch 6/20
33600/33600 [==============================] - 421s - loss: 0.0317 - categorical_accuracy: 0.9900   
Epoch 7/20
33600/33600 [==============================] - 415s - loss: 0.0342 - categorical_accuracy: 0.9884   
Epoch 8/20
33600/33600 [==============================] - 427s - loss: 0.0281 - categorical_accuracy: 0.9906   
Epoch 9/20
33600/33600 [==============================] - 406s - loss: 0.0219 - categorical_accuracy: 0.

In [6]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf 

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import SGD
from keras import metrics


X = train_df[train_df.columns[1:]]
Y = pd.get_dummies(train_df['label'])

X_values = (X.values.astype(np.float64)/256.0).reshape(len(X), 28, 28, 1)
Y_values = Y.values.astype(np.float32)

x_train, x_test, y_train, y_test = train_test_split(X_values, Y_values, 
                                                    test_size = 0.2, random_state=42)


model = Sequential([
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)),
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Dropout(0.25),
    
    Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Dropout(0.25),
    
    Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(1024, activation = "relu"),
    Dropout(0.5),
    Dense(512, activation = "relu"),
    Dropout(0.5),
    Dense(256, activation = "relu"),
    Dense(128, activation = "relu"),
    Dense(32, activation = "relu"),
    Dense(10, activation = "softmax")
])

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), 
              metrics=[metrics.categorical_accuracy])

train_history = model.fit(x_train, y_train, epochs=40, batch_size=500, verbose=1)
train_history = train_history.history

loss_and_metric = model.evaluate(x_train, y_train, batch_size=128, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))
loss_and_metric = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
print("test, loss and metric: {}".format(loss_and_metric))

Epoch 1/40
33600/33600 [==============================] - 476s - loss: 2.0983 - categorical_accuracy: 0.1969   
Epoch 2/40
33600/33600 [==============================] - 439s - loss: 0.9060 - categorical_accuracy: 0.6513   
Epoch 3/40
33600/33600 [==============================] - 444s - loss: 0.3827 - categorical_accuracy: 0.8892   
Epoch 4/40
33600/33600 [==============================] - 470s - loss: 0.1997 - categorical_accuracy: 0.9456   
Epoch 5/40
33600/33600 [==============================] - 456s - loss: 0.1300 - categorical_accuracy: 0.9657   
Epoch 6/40
33600/33600 [==============================] - 436s - loss: 0.1100 - categorical_accuracy: 0.9712   
Epoch 7/40
33600/33600 [==============================] - 468s - loss: 0.0870 - categorical_accuracy: 0.9782   
Epoch 8/40
33600/33600 [==============================] - 423s - loss: 0.0764 - categorical_accuracy: 0.9806   
Epoch 9/40
33600/33600 [==============================] - 444s - loss: 0.0764 - categorical_accuracy: 0.

In [9]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf 

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import SGD
from keras import metrics


X = train_df[train_df.columns[1:]]
Y = pd.get_dummies(train_df['label'])

X_values = (X.values.astype(np.float64)/256.0).reshape(len(X), 28, 28, 1)
Y_values = Y.values.astype(np.float32)

x_train, x_test, y_train, y_test = train_test_split(X_values, Y_values, 
                                                    test_size = 0.2, random_state=42)


model = Sequential([
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)),
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Dropout(0.25),
    
    Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Dropout(0.25),
    
    Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(1024, activation = "relu"),
    Dropout(0.5),
    Dense(512, activation = "relu"),
    Dropout(0.5),
    Dense(256, activation = "relu"),
    Dropout(0.5),
    Dense(128, activation = "relu"),
    Dropout(0.5),
    Dense(32, activation = "relu"),
    Dense(10, activation = "softmax")
])

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), 
              metrics=[metrics.categorical_accuracy])

train_history = model.fit(x_train, y_train, epochs=60, batch_size=500, verbose=1)
train_history = train_history.history

loss_and_metric = model.evaluate(x_train, y_train, batch_size=128, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))
loss_and_metric = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
print("test, loss and metric: {}".format(loss_and_metric))

Epoch 1/60
33600/33600 [==============================] - 463s - loss: 2.2920 - categorical_accuracy: 0.1304   
Epoch 2/60
33600/33600 [==============================] - 446s - loss: 1.8574 - categorical_accuracy: 0.2776   
Epoch 3/60
33600/33600 [==============================] - 446s - loss: 0.7728 - categorical_accuracy: 0.7252   
Epoch 4/60
33600/33600 [==============================] - 431s - loss: 0.3059 - categorical_accuracy: 0.9269   
Epoch 5/60
33600/33600 [==============================] - 439s - loss: 0.2131 - categorical_accuracy: 0.9551   
Epoch 6/60
33600/33600 [==============================] - 433s - loss: 0.1614 - categorical_accuracy: 0.9657   
Epoch 7/60
33600/33600 [==============================] - 438s - loss: 0.1254 - categorical_accuracy: 0.9727   
Epoch 8/60
33600/33600 [==============================] - 440s - loss: 0.1074 - categorical_accuracy: 0.9790   
Epoch 9/60
33600/33600 [==============================] - 431s - loss: 0.1039 - categorical_accuracy: 0.

In [10]:
test_X_values = (test_df.values.astype(np.float64)/256.0).reshape(len(test_df), 28, 28, 1)
test_y_pred = model.predict_classes(test_X_values)

submit_df = pd.DataFrame({"ImageId":range(1, 1+len(test_y_pred)), "Label":test_y_pred})
submit_df.to_csv('test_mnist.csv', index=False)

28000/28000 [==============================] - 106s   
